# Lecture 3: Password Attack with Differential-Power-Analysis (Kocher et al. 1999)

In [ ]:
%load_ext autoreload
%autoreload 2

import os
import random

import numpy as np
import plotly.graph_objects as pgo
from cwtoolbox import CaptureDevice

In [ ]:
capture_device = CaptureDevice.create("CWLITEXMEGA")
capture_device.compile(file=os.path.abspath("sbox_lookup.c"))
capture_device.flash()

In [ ]:
data = capture_device.capture(
    number_of_traces=1000, input=lambda _: [random.randint(0, 255)] + 15 * [0]
)

fig = pgo.Figure()
for d in data[:10]:
    fig.add_trace(pgo.Scatter(y=d["trace"]))
fig.show()

In [ ]:
data_lsb_0 = np.array([d for d in data if d["input"][0] & 0x80 == 0])
data_lsb_1 = np.array([d for d in data if d["input"][0] & 0x80 != 0])

In [ ]:
mean_trace_lsb_0 = np.mean(data_lsb_0["trace"], axis=0)
mean_trace_lsb_1 = np.mean(data_lsb_1["trace"], axis=0)

mean_trace_lsb_0.shape, mean_trace_lsb_1.shape

In [ ]:
fig = pgo.Figure()
fig.add_trace(pgo.Scatter(y=mean_trace_lsb_0 - mean_trace_lsb_1))
fig.show()

In [ ]:
from lascar.tools.aes import sbox


def split(arr, condition):
    return arr[np.array(condition)], arr[~np.array(condition)]

In [ ]:
diff1_1, diff1_2 = split(data, [sbox[d["input"][0] ^ 0x00] & 0x80 == 0 for d in data])
diff2_1, diff2_2 = split(data, [sbox[d["input"][0] ^ 0x01] & 0x80 == 0 for d in data])

fig = pgo.Figure()
fig.add_trace(
    pgo.Scatter(
        y=np.abs(np.mean(diff1_1["trace"], axis=0) - np.mean(diff1_2["trace"], axis=0))
    )
)
fig.add_trace(
    pgo.Scatter(
        y=np.abs(np.mean(diff2_1["trace"], axis=0) - np.mean(diff2_2["trace"], axis=0))
    )
)
fig.show()

In [ ]:
def aes_sbox_dpa(
    data,
    key_byte_index=0,
    selection_bit_index=0,
):
    diffs = []
    for guess in range(256):
        d1, d2 = split(
            data,
            [
                sbox[d["input"][key_byte_index] ^ guess] & (1 << selection_bit_index)
                == 0
                for d in data
            ],
        )
        diffs.append(
            (
                np.max(
                    np.abs(np.mean(d1["trace"], axis=0) - np.mean(d2["trace"], axis=0))
                ),
                guess,
            )
        )
    return sorted(diffs, reverse=True)

In [ ]:
aes_sbox_dpa(data, key_byte_index=0, selection_bit_index=7)